In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pickle

from gpu import set_gpu
import copy as cp
from sklearn.utils import shuffle

import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from DL_utils import MLP, MLPbeta, CNN, eval_nn, train_mlp, test_mlp, train_mlpbeta, test_mlpbeta, train_cnn, test_cnn
import session_new as session

import process_data as prd
from lda import train_lda, predict, eval_lda, eval_lda_ch

set_gpu()

Using TensorFlow backend.


Num GPUs Available:  1


In [2]:
sub_type = 'AB'
with open('train_data_raw_'  + sub_type + '.p', 'rb') as f:
    raw, params,_,_ = pickle.load(f)

n_subs = np.max(params[:,0])

In [4]:
mlp = MLP()
mlp_beta = MLPbeta()
cnn = CNN()

mlp(x_train_mlp)
mlp_beta(x_train_mlp)
cnn(x_train_cnn)

mlp.save_weights('mlp_init_2.h5')
mlp_beta.save_weights('mlpbeta_init_2.h5')
cnn.save_weights('cnn_init_2.h5')

In [3]:
train_sess = {'sub_type':sub_type,'train':'fullallmix4', 'train_grp':2, 'train_scale':5, 'cv_type':'all','scaler_load':False,'feat_type':'feat','epochs':30}
test_sess = {'test_grp':4, 'test':'partposrealmixeven24'}
sess = session.Sess(**train_sess)
sess.update(**test_sess)
n_subs = 4#np.max(params[:,0])

for sub in range(3,n_subs+1):
    sess.sub = sub
    # tf.keras.backend.clear_session()

    if sub > 3:
        mlp.load_weights('mlp_init.h5')
        mlp_beta.load_weights('mlpbeta_init.h5')
        cnn.load_weights('cnn_init.h5')
    else:
    # Train NNs
        mlp = MLP()
        mlp_beta = MLPbeta()
        cnn = CNN()

        loss_fn = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
        optimizer = tf.keras.optimizers.Adam()

        train_loss = tf.keras.metrics.Mean(name='train_loss')
        train_accuracy = tf.keras.metrics.CategoricalAccuracy(name='train_accuracy')

        test_loss = tf.keras.metrics.Mean(name='test_loss')
        test_accuracy = tf.keras.metrics.CategoricalAccuracy(name='test_accuracy')

    # mlp.compile(optimizer='Adam', loss='categorical_crossentropy',metrics=['accuracy'])
    # mlp_beta.compile(optimizer='Adam', loss='categorical_crossentropy',metrics=['accuracy'])
    # cnn.compile(optimizer='Adam', loss='categorical_crossentropy',metrics=['accuracy'])

    train_ind = (params[:,0] == sess.sub) & (params[:,3] == sess.train_grp)
    if np.sum(train_ind):
        trainmlp, validmlp, traincnn, validcnn, y_train, y_valid, x_train_mlp, x_train_cnn, x_train_lda, y_train_lda, x_train_aug, y_train_aug = prd.prep_train_data(sess,raw,params)
        with open('noisedata_all_emgscalelim/AB' + str(sub) + '_grp_2_fullallmix4_5.p','rb') as f:
            scaler, x_train_cnn, _, x_valid_cnn, _, y_train, y_valid, x_train_lda, y_train_lda, x_train_aug, y_train_aug = pickle.load(f)
        
        x_train_cnn = x_train_cnn.astype('float32')
        x_train_mlp = x_train_cnn.reshape(x_train_cnn.shape[0],-1)
        
        trainmlp = tf.data.Dataset.from_tensor_slices((x_train_mlp, y_train)).shuffle(x_train_mlp.shape[0],reshuffle_each_iteration=True).batch(128)
        traincnn = tf.data.Dataset.from_tensor_slices((x_train_cnn, y_train)).shuffle(x_train_cnn.shape[0],reshuffle_each_iteration=True).batch(128)
        
        sess.scaler = scaler

        models = ['mlp','mlpbeta','cnn']

        # mlp.fit(x_train_mlp,y_train,epochs=30,batch_size=128,verbose=0)
        # mlp_beta.fit(x_train_mlp,y_train,epochs=30,batch_size=128,verbose=0)
        # cnn.fit(x_train_cnn,y_train,epochs=30,batch_size=128,verbose=0)

        for model in models:
            print('Training ' + model)
            for epoch in range(sess.epochs):
                # Reset the metrics at the start of the next epoch
                train_loss.reset_states()
                train_accuracy.reset_states()
                test_loss.reset_states()
                test_accuracy.reset_states()

                # Train MLP
                if model == 'mlp':
                    for x, y in trainmlp:
                        train_mlp(x, y, mlp, loss_fn, optimizer, train_loss, train_accuracy)
                    # for x_test, y_test in validmlp:
                    #     test_mlp(x_test, y_test, mlp, loss_fn, test_loss, test_accuracy)
                # Train MLP Beta
                elif model == 'mlpbeta':
                    for x, y in trainmlp:
                        train_mlpbeta(x, y, mlp_beta, loss_fn, optimizer, train_loss, train_accuracy)
                    # for x_test, y_test in validmlp:
                    #     test_mlpbeta(x_test, y_test, mlp_beta, loss_fn, test_loss, test_accuracy)
                # Train CNN
                elif model == 'cnn':
                    for x, y in traincnn:
                        train_cnn(x, y, cnn, loss_fn, optimizer, train_loss, train_accuracy)
                    # for x_test, y_test in validcnn:
                    #     test_cnn(x_test, y_test, cnn, loss_fn, test_loss, test_accuracy)

                if epoch == 0 or epoch == sess.epochs-1:
                    print(
                        f'Epoch {epoch + 1}, '
                        f'Loss: {train_loss.result():.2f}, '
                        f'Accuracy: {train_accuracy.result() * 100:.2f}, '
                        # f'Test Loss: {test_loss.result():.2f}, '
                        # f'Test Accuracy: {test_accuracy.result() * 100:.2f}'
                    )
        
        # Train aligned
        y_train_aligned = np.argmax(y_train, axis=1)[...,np.newaxis]

        mlp_enc = mlp.get_layer(name='enc')
        w_mlp, c_mlp,_, _, _ = train_lda(mlp_enc(x_train_mlp).numpy(),y_train_aligned)

        mlpbeta_enc = mlp_beta.get_layer(name='enc')
        w_mlpbeta, c_mlpbeta,_, _, _ = train_lda(mlpbeta_enc(x_train_mlp).numpy(),y_train_aligned)
        
        cnn_enc = cnn.get_layer(name='enc')
        temp = cnn_enc(x_train_cnn[:x_train_cnn.shape[0]//2,...]).numpy()
        temp2 = np.vstack((temp,cnn_enc(x_train_cnn[x_train_cnn.shape[0]//2:,...]).numpy()))
        w_cnn, c_cnn,_, _, _ = train_lda(temp2,y_train_aligned)

        # Train LDA
        w,c, _, _, _ = train_lda(x_train_lda,y_train_lda)
        w_aug,c_aug, _, _, _ = train_lda(x_train_aug,y_train_aug)

        # Load test data
        with open('real_noise/all_real_noise.p', 'rb') as f:
            real_noise_temp, _ = pickle.load(f)

        test_ind = (params[:,0] == sess.sub) & (params[:,3] == sess.test_grp)

        if np.sum(test_ind):
            x_test_cnn, x_test_mlp, x_test_lda, y_test, clean_size = prd.prep_test_data(sess, raw, params, real_noise_temp)
            with open('testdata_all_emgscalelim_noisescalelim/AB' + str(sub) + '_grp_2_partposrealmixeven14_1.p','rb') as f:
                x_test_cnn, _, x_test_lda, y_test = pickle.load(f) 
            x_temp = np.transpose(x_test_lda.reshape((x_test_lda.shape[0],4,-1)),(0,2,1))[...,np.newaxis]
            x_test_cnn = scaler.transform(x_temp.reshape(x_temp.shape[0]*x_temp.shape[1],-1)).reshape(x_temp.shape)
            x_test_cnn = x_test_cnn.astype('float32')

            # Reshape for nonconvolutional SAE
            x_test_mlp = x_test_cnn.reshape(x_test_cnn.shape[0],-1)
            # Test
            mlp_test_aligned = mlp_enc(x_test_mlp).numpy()
            mlpbeta_test_aligned = mlpbeta_enc(x_test_mlp).numpy()
            cnn_test_aligned = cnn_enc(x_test_cnn).numpy()
            y_test_aligned = np.argmax(y_test, axis=1)[...,np.newaxis]

            clean_lda = eval_lda(w, c, x_test_lda[:clean_size,...], y_test_aligned[:clean_size,...])
            noisy_lda = eval_lda(w, c, x_test_lda[clean_size:,...], y_test_aligned[clean_size:,...])
            print(
                f'LDA ---- '
                f'Clean: {clean_lda * 100:.2f}, '
                f'Noisy: {noisy_lda * 100:.2f}'
            )

            clean_lda = eval_lda(w_aug, c_aug, x_test_lda[:clean_size,...], y_test_aligned[:clean_size,...])
            noisy_lda = eval_lda(w_aug, c_aug, x_test_lda[clean_size:,...], y_test_aligned[clean_size:,...])
            print(
                f'AUG ---- '
                f'Clean: {clean_lda * 100:.2f}, '
                f'Noisy: {noisy_lda * 100:.2f}'
            )

            clean_acc, noisy_acc = eval_nn(x_test_mlp, y_test,mlp,clean_size)
            clean_lda = eval_lda(w_mlp,c_mlp,mlp_test_aligned[:clean_size,...],y_test_aligned[:clean_size,...])
            noisy_lda = eval_lda(w_mlp,c_mlp,mlp_test_aligned[clean_size:,...],y_test_aligned[clean_size:,...])
            print(
                f'MLP ---- '
                f'Clean: {clean_acc * 100:.2f}, '
                f'Noisy: {noisy_acc * 100:.2f}, '
                f'LDA Clean: {clean_lda * 100:.2f}, '
                f'LDA Noisy: {noisy_lda * 100:.2f}'
            )

            clean_acc, noisy_acc = eval_nn(x_test_mlp, y_test,mlp_beta,clean_size)
            clean_lda = eval_lda(w_mlpbeta,c_mlpbeta,mlpbeta_test_aligned[:clean_size,...],y_test_aligned[:clean_size,...])
            noisy_lda = eval_lda(w_mlpbeta,c_mlpbeta,mlpbeta_test_aligned[clean_size:,...],y_test_aligned[clean_size:,...])
            print(
                f'MLPB ---- '
                f'Clean: {clean_acc * 100:.2f}, '
                f'Noisy: {noisy_acc * 100:.2f}, '
                f'LDA Clean: {clean_lda * 100:.2f}, '
                f'LDA Noisy: {noisy_lda * 100:.2f}'
            )

            clean_acc, noisy_acc = eval_nn(x_test_cnn, y_test,cnn,clean_size)
            clean_lda = eval_lda(w_cnn,c_cnn,cnn_test_aligned[:clean_size,...],y_test_aligned[:clean_size,...])
            noisy_lda = eval_lda(w_cnn,c_cnn,cnn_test_aligned[clean_size:,...],y_test_aligned[clean_size:,...])
            print(
                f'CNN ---- '
                f'Clean: {clean_acc * 100:.2f}, '
                f'Noisy: {noisy_acc * 100:.2f}, '
                f'LDA Clean: {clean_lda * 100:.2f}, '
                f'LDA Noisy: {noisy_lda * 100:.2f}'
            )
        else:
            print('no testing data')
    else:
        print('no training data')



Loading training data: traindata_all/AB3_traindata_2.p
Training mlp

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Epoch 1, Loss: 1.77, Accuracy: 32.27, 
Training mlpbeta

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the 

KeyboardInterrupt: 

In [ ]:
Loading training data: traindata_all/AB3_traindata_2.p
Training mlp
Epoch 1, Loss: 1.89, Accuracy: 27.14, 
Epoch 30, Loss: 0.60, Accuracy: 79.52, 
Training mlpbeta
Epoch 1, Loss: 1.07, Accuracy: 62.94, 
Epoch 30, Loss: 0.21, Accuracy: 92.56, 
Training cnn
Epoch 1, Loss: 0.91, Accuracy: 71.01, 
Epoch 30, Loss: 0.17, Accuracy: 93.55, 
Loading training data: traindata_all/AB3_traindata_4.p
LDA ---- Clean: 76.14, Noisy: 48.57
AUG ---- Clean: 58.46, Noisy: 47.11
MLP ---- Clean: 74.49, Noisy: 64.69, LDA Clean: 75.03, LDA Noisy: 64.31
MLPB ---- Clean: 81.80, Noisy: 76.69, LDA Clean: 80.74, LDA Noisy: 78.06
CNN ---- Clean: 74.09, Noisy: 70.54, LDA Clean: 74.00, LDA Noisy: 70.91
Loading training data: traindata_all/AB4_traindata_2.p
Training mlp
Epoch 1, Loss: 1.78, Accuracy: 28.23, 
Epoch 30, Loss: 0.76, Accuracy: 70.97, 
Training mlpbeta
Epoch 1, Loss: 1.37, Accuracy: 50.24, 
Epoch 30, Loss: 0.36, Accuracy: 86.45, 
Training cnn
Epoch 1, Loss: 1.41, Accuracy: 44.92, 
Epoch 30, Loss: 0.32, Accuracy: 88.22, 
Loading training data: traindata_all/AB4_traindata_4.p
LDA ---- Clean: 86.49, Noisy: 52.31
AUG ---- Clean: 50.91, Noisy: 44.46
MLP ---- Clean: 81.17, Noisy: 74.23, LDA Clean: 84.43, LDA Noisy: 75.57
MLPB ---- Clean: 81.63, Noisy: 76.11, LDA Clean: 83.11, LDA Noisy: 78.14
CNN ---- Clean: 82.69, Noisy: 75.34, LDA Clean: 84.51, LDA Noisy: 77.23

In [ ]:
tape shuffle
Loading training data: traindata_all/AB2_traindata_2.p
Loading training data: traindata_all/AB2_traindata_4.p
LDA ---- Clean: 90.23, Noisy: 39.63
AUG ---- Clean: 56.11, Noisy: 45.74
MLP ---- Clean: 80.66, Noisy: 73.31, LDA Clean: 80.66, LDA Noisy: 72.86
MLPB ---- Clean: 91.17, Noisy: 78.11, LDA Clean: 90.57, LDA Noisy: 78.23
CNN ---- Clean: 91.11, Noisy: 77.63, LDA Clean: 93.89, LDA Noisy: 80.77
Loading training data: traindata_all/AB3_traindata_2.p
Loading training data: traindata_all/AB3_traindata_4.p
LDA ---- Clean: 76.14, Noisy: 49.54
AUG ---- Clean: 57.69, Noisy: 45.71
MLP ---- Clean: 64.17, Noisy: 58.89, LDA Clean: 69.77, LDA Noisy: 61.26
MLPB ---- Clean: 76.51, Noisy: 73.29, LDA Clean: 76.57, LDA Noisy: 72.91
CNN ---- Clean: 66.86, Noisy: 63.11, LDA Clean: 71.66, LDA Noisy: 67.29
Loading training data: traindata_all/AB4_traindata_2.p
Loading training data: traindata_all/AB4_traindata_4.p
LDA ---- Clean: 86.49, Noisy: 52.06
AUG ---- Clean: 51.26, Noisy: 45.43
MLP ---- Clean: 75.37, Noisy: 65.43, LDA Clean: 73.60, LDA Noisy: 64.09
MLPB ---- Clean: 85.80, Noisy: 78.26, LDA Clean: 83.66, LDA Noisy: 77.89
CNN ---- Clean: 77.94, Noisy: 73.23, LDA Clean: 83.57, LDA Noisy: 75.97

In [ ]:
tape no shuffle
Loading training data: traindata_all/AB2_traindata_2.p
Loading training data: traindata_all/AB2_traindata_4.p
LDA ---- Clean: 90.23, Noisy: 39.66
AUG ---- Clean: 56.91, Noisy: 46.77
MLP ---- Clean: 80.77, Noisy: 69.49, LDA Clean: 78.09, LDA Noisy: 67.66
MLPB ---- Clean: 89.34, Noisy: 74.29, LDA Clean: 88.66, LDA Noisy: 74.71
CNN ---- Clean: 88.34, Noisy: 77.51, LDA Clean: 91.03, LDA Noisy: 79.26
Loading training data: traindata_all/AB3_traindata_2.p
Loading training data: traindata_all/AB3_traindata_4.p
LDA ---- Clean: 76.14, Noisy: 49.29
AUG ---- Clean: 58.09, Noisy: 46.06
MLP ---- Clean: 66.46, Noisy: 60.09, LDA Clean: 67.71, LDA Noisy: 61.11
MLPB ---- Clean: 74.23, Noisy: 73.63, LDA Clean: 79.57, LDA Noisy: 75.03
CNN ---- Clean: 65.20, Noisy: 62.77, LDA Clean: 67.71, LDA Noisy: 65.40
Loading training data: traindata_all/AB4_traindata_2.p
Loading training data: traindata_all/AB4_traindata_4.p
LDA ---- Clean: 86.49, Noisy: 53.06
AUG ---- Clean: 50.54, Noisy: 45.23
MLP ---- Clean: 61.31, Noisy: 57.29, LDA Clean: 61.71, LDA Noisy: 57.91
MLPB ---- Clean: 83.89, Noisy: 77.69, LDA Clean: 84.91, LDA Noisy: 78.60
CNN ---- Clean: 84.49, Noisy: 79.00, LDA Clean: 86.03, LDA Noisy: 78.51

In [ ]:
model fit shuffle
Loading training data: traindata_all/AB2_traindata_2.p
Loading training data: traindata_all/AB2_traindata_4.p
LDA ---- Clean: 90.23, Noisy: 40.06
AUG ---- Clean: 56.00, Noisy: 45.74
MLP ---- Clean: 79.26, Noisy: 70.54, LDA Clean: 79.54, LDA Noisy: 70.37
MLPB ---- Clean: 90.31, Noisy: 78.31, LDA Clean: 90.31, LDA Noisy: 79.89
CNN ---- Clean: 89.94, Noisy: 75.54, LDA Clean: 89.14, LDA Noisy: 75.63
Loading training data: traindata_all/AB3_traindata_2.p
Loading training data: traindata_all/AB3_traindata_4.p
LDA ---- Clean: 76.14, Noisy: 49.69
AUG ---- Clean: 57.46, Noisy: 46.23
MLP ---- Clean: 77.43, Noisy: 65.29, LDA Clean: 77.43, LDA Noisy: 70.29
MLPB ---- Clean: 83.83, Noisy: 78.54, LDA Clean: 81.23, LDA Noisy: 77.60
CNN ---- Clean: 75.00, Noisy: 69.09, LDA Clean: 74.00, LDA Noisy: 69.26
Loading training data: traindata_all/AB4_traindata_2.p
Loading training data: traindata_all/AB4_traindata_4.p
LDA ---- Clean: 86.49, Noisy: 52.60
AUG ---- Clean: 50.91, Noisy: 44.31
MLP ---- Clean: 82.00, Noisy: 72.91, LDA Clean: 80.69, LDA Noisy: 72.29
MLPB ---- Clean: 80.80, Noisy: 75.74, LDA Clean: 81.86, LDA Noisy: 77.63
CNN ---- Clean: 85.86, Noisy: 79.29, LDA Clean: 84.94, LDA Noisy: 79.94

In [ ]:
model fit no shuffle
Loading training data: traindata_all/AB2_traindata_2.p
Loading training data: traindata_all/AB2_traindata_4.p
LDA ---- Clean: 90.23, Noisy: 40.11
AUG ---- Clean: 56.57, Noisy: 45.71
MLP ---- Clean: 77.11, Noisy: 68.03, LDA Clean: 78.86, LDA Noisy: 69.00
MLPB ---- Clean: 86.09, Noisy: 75.20, LDA Clean: 90.03, LDA Noisy: 77.83
CNN ---- Clean: 88.20, Noisy: 76.83, LDA Clean: 89.46, LDA Noisy: 77.54
Loading training data: traindata_all/AB3_traindata_2.p
Loading training data: traindata_all/AB3_traindata_4.p
LDA ---- Clean: 76.14, Noisy: 49.66
AUG ---- Clean: 57.54, Noisy: 46.34
MLP ---- Clean: 72.97, Noisy: 66.71, LDA Clean: 79.11, LDA Noisy: 71.26
MLPB ---- Clean: 73.03, Noisy: 70.89, LDA Clean: 76.94, LDA Noisy: 73.74
CNN ---- Clean: 73.83, Noisy: 70.54, LDA Clean: 73.83, LDA Noisy: 68.31
Loading training data: traindata_all/AB4_traindata_2.p
Loading training data: traindata_all/AB4_traindata_4.p
LDA ---- Clean: 86.49, Noisy: 52.06
AUG ---- Clean: 51.06, Noisy: 44.74
MLP ---- Clean: 75.97, Noisy: 66.54, LDA Clean: 76.23, LDA Noisy: 65.86
MLPB ---- Clean: 77.60, Noisy: 72.11, LDA Clean: 81.29, LDA Noisy: 75.17
CNN ---- Clean: 86.46, Noisy: 78.54, LDA Clean: 85.00, LDA Noisy: 78.03